In [1]:
from pyDatabases.gpyDB.gpyDBnew import *

*NOTE: It is pretty slow to initialize two important GAMS related parts of the database. The ```gams.GamsWorkspace``` and the ```gams.core.numpy.gams2numpy.Gams2Numpy``` classes. This means that everytime we initialize a database with new instances of the two, it takes roughly 1/4 second. Thus, it can be quite advantageous not to initialize new instances of the two too often.*

## 1. GpyDB tests

Default working folder and data folders:

In [2]:
work_folder = os.path.join(os.getcwd(),'workFolder')
data_folder = os.path.join(os.getcwd(), 'testdata')

### 1.1. Init methods

We can initialize the database in several ways. The init methods are implemented in a multiple-dispatch like fashion, so it shouldn't impede the speed of initialization. The following covers the basics:

#### i. From GamsDatabase

To load a GamsDatabase, we need a ```gams.Workspace```. We can add a new one using the staticmethod:

In [3]:
ws = GpyDB.initWs(work_folder)

Next, let us say that we already have a GamsDatabase open (here we load one from a gdx file):

In [4]:
name = 'testdb'
gdxStr = os.path.join(data_folder,f'{name}.gdx')
dbGms = ws.add_database_from_gdx(gdxStr, database_name = name)

We can initialize the GpyDB as a wrapper around  this database by calling:

In [5]:
db = GpyDB(dbGms, alias = None, data_folder = data_folder, dropattrs = None)

In this case, we can only add the options (defaults in parenthesis):
* ```alias = None```: Update specification of aliases (we'll return to this later).
* ```data_folder = os.getcwd()```: Specification of where to store data if exporting it.
* ```dropattrs = ['database','ws','g2np','gmd'].copy()```: What attributes of the class not to include when exporting it.

We note that a number of attributes are set automatically: These cannot be adjusted in the ```__init__``` call, but may be adjusted later:
* ```self.name```: Inherited from the GamsDatabase.
* ```self.ws```: Inherited from the GamsDatabase.
* ```self.g2np```: Instance of ```gams.core.numpy.gams2numpy.Gams2Numpy``` class that is required when working with gams data. To save time when initializing, all instances rely on the same instance of this class that is created when we import the packag. *Note: You need to adjust this instance, if you do not want to work in the default installation of GAMS (if you have more than one). See the online documentation for the GAMS Python API for more.*
* ```self.gmd```: Instance of ```gpyDB.readGmd``` that is used when importing/exporting data from GAMS to Python.
* ```self.series```: ```gpyDB.SeriesDB``` instance that is read from the ```GamsDatabase```.

We export this as a pickle (here using default options):

In [6]:
db.export(name = None, repo = None) # name defaults to self.name, repo defaults to self.data_folder

Note that, when exporting data, we automatically drop the attributes in:

In [7]:
db.dropattrs

['database', 'ws', 'g2np', 'gmd']

The default behavior here is to keep the data stored in the python version from ```self.series``` instead of ```self.database```. If we insist on keeping the gdx data, we can always write:

In [8]:
# db.dropattrs = [k for k in db.dropattrs if k != 'database']

If we do this, any call to the pickle module will start by writing a gdx file with the database and store it in the specified data folder. Then, when loading the pickle again, the database will be read in to the ```self.database``` attribute. This is a workaround that is required as the gams instances are generally not pickleable (Python does not automatically know how to store it).

#### ii. From Pickle 

We can load the GpyDB from the pickled data using the path for the pickle. The only thing that we can adjust in the init call when using the pickled data is whether to adjust the workspace and aliases. In this case, we open the data in the existing workspace:

*Note: The pickled data should always be loaded using the ```__init__``` method as outlined here - not using the standard load method from the pickle module (this is called "under the hood").*

In [9]:
pcklStr = os.path.join(data_folder, name)
dbPickled = GpyDB(pcklStr, ws = ws)

Note that the pickle will try to open the database with the name ```testdb``` (as specified in the initialization). However, as we also want to reuse the GamsWorkspace, we will automatically get a "versionized name" for the database (GAMS requires unique names for all databases in the workspace):

In [10]:
dbPickled.name

'testdb__v1'

In [11]:
db.__dict__

{'database': <gams.control.database.GamsDatabase at 0x1cd9cecd950>,
 'ws': <gams.control.workspace.GamsWorkspace at 0x1cd9cecc750>,
 'work_folder': 'C:\\Users\\sxj477\\Documents\\GitHub\\pyDatabases\\workFolder',
 'name': 'testdb',
 'g2np': <gams.core.numpy.gams2numpy.Gams2Numpy at 0x1cd9cabee90>,
 'data_folder': 'C:\\Users\\sxj477\\Documents\\GitHub\\pyDatabases\\testdata',
 'dropattrs': ['database', 'ws', 'g2np', 'gmd'],
 'gmd': <pyDatabases.gpyDB.database.readGmd at 0x1cd9ae77290>,
 'series': <pyDatabases.gpyDB.gpyDBnew.SeriesDB at 0x1cd99ec28d0>}

#### iii. From a GpyDB

We can also initialize it from another ```GpyDB```. This creates a copy of all attributes *except*  three gams related attributes ```ws, gmd, database```. If we do not provide a workspace, this inherits the one from the other database. The ```gmd``` and ```database``` attributes are then copied within this workspace:

In [12]:
dbFromGpyDB = GpyDB(db) # create copy of all except the four gams-related attributes

*The workspace is now inherited:*

In [13]:
dbFromGpyDB.ws is db.ws

True

*The self.database is a copy - and the name is versionized to exist in the same workspace:*

In [14]:
dbFromGpyDB.name, dbFromGpyDB.database is db.database # versionized name and copy of the database

('testdb__v2', False)

Using the ```ws``` option, we can e.g. start in a new workspace:

In [15]:
dbFromGpyDB_newWs = GpyDB(db, ws = work_folder) # copy GpyDB in new workspace
dbFromGpyDB_newWs.ws is db.ws

False

*Note: The method ```self.copy(ws = None, **kwargs)``` relies on this exact method to produce a copy:*

In [16]:
copiedDatabase = db.copy(ws = dbFromGpyDB_newWs.ws)
copiedDatabase.ws is dbFromGpyDB_newWs.ws

True

#### iv. From other database types:

Alternatively, we can initialize the GpyDB with information e.g. on the series or database attributes. This is done by passing ```obj=None``` as the first input and instead providing a ```db``` input. Beyond this argument, we can provide the following kwargs:
* ```name```: Name of the ```GpyDB``` instance.
* ```ws```: Works as in the other init types (can either reference an existing workspace, None (defaults to ```os.getcwd()``` then), or a string (indicating a repository to use).
* ```data_folder```: where to export stuff.
* ```dropattrs```: what to drop before exporting.

In [17]:
dbFromSeries = GpyDB(obj = None, db = db.series, ws = db.ws, name = 'NewName')

Note that the this series is not copied but added directly as the ```self.series``` database, i.e. symbols are now the same across the two databases:

In [18]:
dbFromSeries['i'] is db['i']

True

We can also do this with a simple ```dict``` database with gpy symbols, for instance:

In [19]:
dbFromDict = GpyDB(obj = None, db = db.series.database, ws = db.ws, name = 'NewName')

Finally, we can pass a ```GamsDatabase``` to this argument. Recall that if we passed this type as the ```obj``` argument, the ```GpyDB``` "wrapped" around this, i.e. it draws on this database directly as ```self.database```. If we use ```db = GamsDatabase``` instead, we open a new database drawing on this as a source:

In [20]:
dbFromGams_sourceDB = GpyDB(obj = None, db = db.database, ws = db.ws, name = 'otherName')
dbFromGams_sourceDB.database is db.database # if we initialized with obj = db.database this would be true

False

### 1.2. Merge, export, etc.

*Merge internal with ```priority = 'second'```:*

In [21]:
db.mergeInternal(priority = 'second') # overwrite existing symbols
db.database.export('test1.gdx') # test that exports look as expected

*Merge internal with ```merge = 'replace'```: This creates a new ```GamsDatabase``` in the workspace, where everything is added to:*

In [22]:
db.mergeInternal(priority = 'replace', name = 'test2')
db.database.export('test2.gdx')
db.ws._gams_databases # the original workspace still contains the original database, but the main db in the GpyDB class is the new one 'test2__v1'.

{'testdb': 0,
 'testdb__v1': 0,
 'testdb__v2': 0,
 'NewName': 0,
 'NewName__v1': 0,
 'otherName': 0,
 'test2': 0}

*Test pickle with gdx data included:*

In [23]:
db.dropattrs = [k for k in db.dropattrs if k !='database'] # remove 'database' from list of attributes NOT to keep
db.export(name = 'GpyDBwithGdx') # export with gdx database

*As the gdx database is not pickleable, it is written to a separate file in the data folder and loaded again if we pickle this:*

In [24]:
dbPickled_v2 = GpyDB(os.path.join(db.data_folder, 'GpyDBwithGdx'))
dbPickled_v2.gmd.symbols # symbols from database

{'alias_set': <Swig Object of type 'void *' at 0x000001CD9CECB3F0>,
 'alias_map2': <Swig Object of type 'void *' at 0x000001CD9CECBD20>,
 'i': <Swig Object of type 'void *' at 0x000001CD9CECA280>,
 'j': <Swig Object of type 'void *' at 0x000001CD9CECAEE0>,
 'alias_': <Swig Object of type 'void *' at 0x000001CD9CECBA50>,
 'map': <Swig Object of type 'void *' at 0x000001CD9CECB330>,
 'var': <Swig Object of type 'void *' at 0x000001CD9CECB7B0>,
 'var1d': <Swig Object of type 'void *' at 0x000001CD9CECAE50>,
 'param': <Swig Object of type 'void *' at 0x000001CD9CECA490>,
 'scalar': <Swig Object of type 'void *' at 0x000001CD9CECBF00>,
 'pscalar': <Swig Object of type 'void *' at 0x000001CD9CEC8F00>,
 'subset': <Swig Object of type 'void *' at 0x000001CD9CEC8EA0>}

### 1.3. Methods

#### Properties

In [25]:
db.aliasDict # map from original set to aliases
# db.aliasDict0 # includes the original set in the values

{'j': Index(['jj'], dtype='object', name='alias_map2')}

In [26]:
db.alias_notin_db # what aliases are not also included in the database as symbols

{'jj'}

#### Get/set

Add symbols to the ```self.series``` database using ```self.__setitem__``` method. This automatically calls the ```gpy``` class, so we can set items either using their pandas/python pendant or the gpy symbols:

In [27]:
db['newSet'] = pd.Index([0,1], name = 'newSet')
db['newSubset'] = pd.Index([0], name = 'newSet') # If the domain of the index does not coincide with the given name in the database --> subset
db['newMap'] = pd.MultiIndex.from_product([db('j'), db('newSet')]) # the .__call__() method accesses the pandas/python part of the symbol directly
[db[k].type for k in ('newSet','newSubset','newMap')]

['set', 'subset', 'map']

The ```self.__call__``` method accesses the ```self.vals``` part of the ```gpy``` symbols, i.e.:

In [28]:
db['newSet'].vals is db('newSet')

True

The ```self.__call__``` and ```self.__getitem__``` methods allows us to access aliased sets as well. For instance, the alias ```'jj'``` is not defined as a symbol in the ```self.series``` database, but calling this we access the original set ```'j'```:

In [29]:
db['jj'] is db['j'] # the self.__getitem__ method accesses the parent set

True

In [30]:
db('jj') # the get method accesses parent set + adjusts name of index

Index([  1,   0,   2,   3,   4,   5,   6,   7,   8,   9,
       ...
       990, 991, 992, 993, 994, 995, 996, 997, 998, 999],
      dtype='int32', name='jj', length=1000)

#### Alias methods

The method ```self.alias(x, idx = 0)```: 
* If ```x``` is not aliased, but a set and ```idx=0```: ```return x```.
* If ```x``` is either an aliased set or a parent set to one, this returns the parent set when ```idx = 0``` as default, while for ```idx>0``` this returns the aliased sets (idx is passed to ```self.aliasDict0```:

In [31]:
db.alias('i') # 'i' is a set, so this works even though it is not aliased

'i'

In [32]:
try:
    db.alias('i', idx = 1)
except TypeError:
    print('This trows a TypeError, because i is not alised')

This trows a TypeError, because i is not alised


In [33]:
'j' == db.alias('j') == db.alias('jj') # aliased sets return the same object no matter if it is a parent set or the aliased one

True

In [34]:
'jj' == db.alias('j',idx = 1) == db.alias('jj',idx=1) # if idx = 1 this returns the first aliased set

True

#### Other

The method ```self.getTypes(types)``` returns a subset of the ```self.symbols``` property based on the types (iterative arg.) that are passed:

In [35]:
db.getTypes(['set']) # get sets

{'alias_set': <pyDatabases.gpyDB.database.gpy at 0x1cd9cf3b0d0>,
 'alias_map2': <pyDatabases.gpyDB.database.gpy at 0x1cd9a25e350>,
 'i': <pyDatabases.gpyDB.database.gpy at 0x1cd9cf45450>,
 'j': <pyDatabases.gpyDB.database.gpy at 0x1cd9cf17fd0>,
 'newSet': <pyDatabases.gpyDB.database.gpy at 0x1cd9cf7ea50>}

In [36]:
db.getTypes(['set','scalarVar']) # get scalar variables

{'alias_set': <pyDatabases.gpyDB.database.gpy at 0x1cd9cf3b0d0>,
 'alias_map2': <pyDatabases.gpyDB.database.gpy at 0x1cd9a25e350>,
 'i': <pyDatabases.gpyDB.database.gpy at 0x1cd9cf45450>,
 'j': <pyDatabases.gpyDB.database.gpy at 0x1cd9cf17fd0>,
 'scalar': <pyDatabases.gpyDB.database.gpy at 0x1cd9cf17090>,
 'newSet': <pyDatabases.gpyDB.database.gpy at 0x1cd9cf7ea50>}

The method ```self.domainsUnique(x)``` returns the unique domains a symbol is defined over, replacing aliased sets with their parent sets:

In [37]:
db.domainsUnique('var')

['i', 'j']

The method ```self.varDom(set_, types = None)``` returns a dictionary with keys = sets and aliases and values = list of symbols of 'types' that are defined over these:

In [38]:
db.varDom('i') # the default is to look for types ['var','par']. 

{'i': ['var', 'var1d', 'param']}

In [39]:
db.varDom('i', types = ['var','map'])

{'i': ['map', 'var', 'var1d']}

## 2. Other classes from ```gpyDB```

### 2.1. ```DbFromExcel```

Small calss that reads in excel data as dicts of ```gpy``` symbols. Data has to be arranged in a particular way for this to work. Examples are given in the ```test_read.xlsx``` file:

In [40]:
file = os.path.join(os.getcwd(), 'testdata','test_read.xlsx')
kwargs = {k: f'_{k}' for k in ('set','subset','map','var','scalarVar','var2D')}
db_ = DbFromExcel.dbFromWB(file, kwargs)

The kwargs indicate key = method to apply when reaing the data, value = sheet or list of sheets to iterate through. The syntax for loading several sheets calling the same method is done by adding:

In [41]:
kwargs['set'] = ['_set','extraset'] # iterable of sheetnames
db_ = DbFromExcel.dbFromWB(file, kwargs)

### 2.2. ```AggDB```

Small class that can be used to aggregate the database. The class contains a handfull of methods that are called when doing the "main" aggregation method.

*Re-read the clean database version:*

In [42]:
db = GpyDB(name = 'test1', db = gdxStr, ws = work_folder, data_folder = data_folder)

To make sure that our alias method work, define a couple of extra symbols over the alias as well:

In [44]:
jj = pd.Index(np.roll(db('map').get_level_values('j').values, 1), name = 'jj')
db['mapWithAlias'] = pd.MultiIndex.from_arrays([db('map').get_level_values(0), db('map').get_level_values(1), jj]).sort_values()
db['subsetWithAlias'] = jj[0:2].rename('subsetWithAlias')
db['varWithAlias_1'] = pd.Series(1.5, index = db('mapWithAlias')) 
db['varWithAlias_2'] = pd.Series(0.5, index = adj.AdjAliasInd(db('map'), alias = {'j':'jj'})[0:15])

#### 2.2.1. Update set elemetns

The first method takes a mapping from old to new set values and updated the values in indices used throughout the database. We note that this is generally *not* used to aggregate databases, i.e. we prefer not to use a mapping that reduces the number of set elements, but merely renames them. However, the method does work even if this is the case. For instance:

In [45]:
setName = 'j'
ns = {1: 0, 2:0, 3:0}

There are two things to be aware of here: 
* We may use ```rul = True``` if we want to remove unused levels in all indices before mapping to the new values (see ```pd.Index.remove_unused_levels```).
* If the namespace mapping (```ns```) that we apply implies non-unique combinations, we generally ensure that all sets, subsets, and mappings are forced to unique combinations, whereas we keep all records of parameters and variables.

In [46]:
db_newSetValues = AggDB.updSetElements(db.copy(), setName, ns)

So, the mapping 'map' includes fewer elements than before:

In [47]:
len(db('map'))-len(db_newSetValues('map'))

3

Whereas the variable 'var' that is defined over the same mapping does not:

In [48]:
len(db('var'))-len(db_newSetValues('var'))

0

#### 2.2.2. Update set names

This method changes the set name used throughout all symbols. For instance, let us say that we want to rename the set $i$ to $k$:

In [49]:
ns = {'i':'k'}
db_newSetNames = AggDB.updSetNames(db.copy(), ns)

#### 2.2.3. Update sets from symbols

This infer set values from the symbols that are otherwise defined in the database. This can be used if we load information e.g. on variables or parameters, and subsequently want to define the underlying sets as well (e.g. because GAMS usually requires this):

In [50]:
db_updSetsFromSyms = db.copy() # create copy
AggDB.updSetsFromSyms(db, types = ['var','par'])

We have the following options when doing this:
* ```types = None```: The type of symbols to infer set definitions from. When ```None``` this defaults to ```['var','par']``` definitions.
* ```clean = True```: If ```True```, it starts by empyting all set, subset, and map definitions.
* ```clean_alias = True```: Reset alias settings (stored in ```self.db['alias_']``` multiindex).
* ```ignore_alias = False```: If ```False```, this does not add aliased sets to the database

#### 2.2.4. Subset database

This adjusts all symbols in the database based on an index, we provide. For instance, say that we want to exclude some elements from the $j$ index across all symbols:

In [51]:
db_subsetDB = AggDB.subsetDB(db.copy(), pd.Index(db('j')[0:3])) # remove all but the first 4 elements in the 'j' index - for all symbols in the database
db_subsetDB('var') # all symbols including e.g. variables are "subsetted" in this manner

i  j
1  1    10.0
   0    10.0
   2    10.0
Name: var, dtype: float64

#### 2.2.5. Aggregate database

This aggregates all symbols according to a mapping from old set elements to new ones. Contrary to the method ```updSetElements```, this method is designed to change the number of set elements in the various indices, and to handle non-unique combinations of variable entries etc..

Say, for instance, that we want to aggregate the set $j$ into intervals of 10's:

In [52]:
mapping = pd.MultiIndex.from_arrays([db('j'), np.round(db('j')+5.1, -1).astype(int).rename('agg_j')])

We need to decide what to do with the variables that are now aggregated. Currently, we have implemented a handful of methods: ```Sum, Mean, WeightedSum, SplitDistr, Lambda```. As a default, the method will use the simple ```Sum``` method. We can adjust this by specifying a dictionary as follows:

In [53]:
aggLike = {'var': {'func': 'Mean', 'kwargs': {}}}
db_agg = AggDB.aggDB(db.copy(), mapping, aggBy = None, replaceWith = None, aggLike = aggLike) # default options

We can also use this, in principle, to disaggregate a database. Let us, for simplicity, assume that we want to split up a single set element in the $j$ set:

In [54]:
mapping = pd.MultiIndex.from_arrays([db('j'), db('j').rename('disagg_j')])

## 3. Methods from ```gpyDB.database.py```

This includes classes that transfers data between Python (```gpy```) and GAMS (```GamsDatabase, Gmd```). Most notably, we have the following classes:
1. ```gpy```: Customized symbol class that ```GpyDB``` draws on.
2. ```readGmd``` (or ```gpyFromGmd```): Transfer data from ```GamsDatabase``` to ```gpy```-like symbols. Draws on the fast methods from ```gams.core.gmd```.
3. ```gmdFromGpy```: Transfer data from ```gpy``` format to ```GamsDatabase``` symbols. Draws on the fast methods from ```gams.core.gmd```.
4. ```gpyFromGt```: Transfer data using ```gams.transfer.Container``` to ```gpy```-like symbols. Pretty fast (not as fast as ```readGmd``` though). 
5. ```gtFromGpy```: Populate a ```gams.transfer.Container``` database using ```gpy``` symbols.
6. ```MergeSyms```: Merge symbols from gams or gpy databases.

As GAMS models rely on ```GamsDatabase```, however, we will rarely use classes 5-6.

Merge methods:

```python
class readGmd:
```

Methods for reading a gams database (```self.database``` from the ```GpyDB``` instance) to dict of ```gpy``` symbols. This is class that is used when initializing ```GpyDB``` from a GamsDatabase.
* The ```self.__call__``` method returns a dictionary of ```gpy``` symbols that represents the entire database.
* The ```self.gpy(symbol)``` method returns a ```gpy``` from the symbol input (needs to be a ```Swig Object```). These symbols can be accessed in dictionary from ```self.symbols```.

```python
class gpyFromGt:
```

Does the same thing as ```readGmd```, but for a ```gams.transfer.Container``` database.

```python
class gtFromGpy:
```

Inverse to ```gpyFromGt```.

```python
class gmdFromGpy
```

Inverse methods to ```readGmd``` class. This has method for adding symbols to GamsDatabases, adjusting them, and adding-or-merging.
* ```self.db(dbGpy, dbGmd, g2np, merge = True)```: Merge symbols from dbGpy (populated with ```gpy``` symbols) into dbGmd (```GamsDatabase```).
* ```self.initDb(dbGpy, dbGmd, g2np)```: Like ```self.db```, but it assumes that ```dbGmd``` is empty / does not have any of the symbols from ```dbGpy``` before doing so.
* ```init(symbol, db)```, ```add(symbol, db, g2np)```, ```adjust(symbol, db, g2np)```: Static methods that initialize, add, and adjust symbols in a database.

### 3.1. MergeSyms

The class is set up to merge symbols from gpy or gmd database types. The method ```MergeSyms.merge``` is a robust version that works with the four combinations. 

**Note that all these methods work *inplace* meaning that you will alter the symbols along the way**. (*This is by design, as the symbols from the gmd database only exists in conjunction with the database itself - thus returning a copy is meaningless unless we initialize an entire new database*).

*Merge two gpy symbols:*

In [55]:
MergeSyms.merge(db['var'], db['param'], priority = 'first') # merge the two, and keep values from the first symbol if they overlap
# MergeSyms.merge(db['var'], db['param'], priority = 'second') # merge the two, and keep values from the second symbol if they overlap
# MergeSyms.merge(db['var'], db['param'], priority = 'replace') # merge the two and only use values from the second symbol

*Merge gpy and gmd symbol:*

In [56]:
MergeSyms.merge(db['var'], db.gmd, name = 'var', priority = 'first') # merge the two and keep values from gpy symbol if they overlap
# MergeSyms.merge(db['var'], db.gmd, name = 'var', priority = 'second') # merge the two and keep values from gmd symbol if they overlap
# MergeSyms.merge(db['var'], db.gmd, name = 'var', priority = 'replace') # merge the two and only use values from the second symbol 

*Merge gmd and gpy symbols:*

In [57]:
MergeSyms.merge(db.gmd, db['var'], name = 'var', priority = 'first')
# MergeSyms.merge(db.gmd, db['var'], name = 'var', priority = 'second')
# MergeSyms.merge(db.gmd, db['var'], name = 'var', priority = 'replace')

*Merge gmd and gmd symbols:*

In [58]:
MergeSyms.merge(db.gmd, db.gmd, name = 'var', priority = 'first')
# MergeSyms.merge(db.gmd, db.gmd, name = 'var', priority = 'second')
# MergeSyms.merge(db.gmd, db.gmd, name = 'var', priority = 'replace')

### 3.2. MergeDbs

The class merges databases that are of the types ```GpyDB, SeriesDB, dict, readGmd``` (the dict should be a dictionary of ```gpy``` symbols). Once again, the method ```MergeDbs.merge``` is a version that works with combinations of the four. As with ```MergeSyms``` these methods all work inplace as well, i.e. we alter the databases.

*This is a very simple test of whether the merges work, as they are all essentially the same database (almost at least, the code above may have altered them slightly)*

In [59]:
dbs = [db, db.series, db.series.database, db.gmd]
for dbi, dbii in itertools.product(dbs, dbs):
    MergeDbs.merge(dbi,dbii)

## 4. Methods from ```auxfuncs.py```

This section is loaded in all other modules and contains basic methods. It includes:
* ```OrdSet``` class: Custom class with ordered sets.
* ```adj``` class: Provides methods to (i) subset pandas with nested conditions, (ii) adjust indices with lags or aliased sets.
* ```adjMultiIndex``` class: Methods used to adjust symbols defined over multiindices. This includes (i) broadcasting, (ii) applying multiindices to symbols that matches/expands sets they are defined over, (iii) appending symbols with pre-specified grids.
* ```readSetsFromDb``` method: Reads in and defines sets/indices based on the ```pd.Index``` objects that variables/mappings are defined over.